In [1]:
import pandas as pd

# Merge and filter data
-  Read all the sensor data
- Filter for only data from sensors present in our model
- Propperly read the timestamps
- Aggregate all the processed data into a new dataframe

In [3]:
# File names and relative paths
# files = ['../Dados/2013AEDL.csv', '../Dados/1S2014AEDL.csv', '../Dados/2S2014AEDL.csv', '../Dados/1P2015AEDL.csv', '../Dados/2P2015AEDL.csv']
files = ['./Dataset/2013AEDL.csv', './Dataset/1S2014AEDL.csv', './Dataset/2S2014AEDL.csv', './Dataset/1P2015AEDL.csv', './Dataset/2P2015AEDL.csv']

# Define the list of id of the sensors in our model
target_ids = [121726,121727,121731,121732,121733,121734,121735,121736,121741,121742,121754,121755,121756]

# Filter the DataFrame: we want to keep only rows where 'EQUIPMENTID' matches the sensor IDs we use
col = 'EQUIPMENTID'

# Create dataframe to concatenate all the useful information
filtered_df = pd.DataFrame()

for file in files:
    df = pd.read_csv(file)
    # df.info()

    # Convert data types before filtering
    df["AGG_PERIOD_START"] = pd.to_datetime(df["AGG_PERIOD_START"])
    df["LANE_BUNDLE_DIRECTION"] = df["LANE_BUNDLE_DIRECTION"].astype("string")
    
    # Filter for only rows with matching sensor ids
    matching_df = df[df[col].isin(target_ids)]
    
    filtered_df = pd.concat([filtered_df, matching_df], ignore_index=True)

    print(f"Original rows: {len(df)}")
    print(f"Filtered rows: {len(matching_df)}")
    # print(filtered_df.info())

filtered_df.info()

Original rows: 3890630
Filtered rows: 2063480
Original rows: 2251668
Filtered rows: 1153163
Original rows: 2745968
Filtered rows: 1312094
Original rows: 888191
Filtered rows: 436965
Original rows: 3881466
Filtered rows: 1917689
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6883391 entries, 0 to 6883390
Data columns (total 20 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   AGGREGATE_BY_LANE_BUNDLEID  int64         
 1   AGG_ID                      int64         
 2   EQUIPMENTID                 int64         
 3   AGG_PERIOD_START            datetime64[ns]
 4   AGG_PERIOD_LEN_MINS         int64         
 5   NR_LANES                    int64         
 6   LANE_BUNDLE_DIRECTION       string        
 7   TOTAL_VOLUME                int64         
 8   AVG_SPEED_ARITHMETIC        float64       
 9   AVG_SPEED_HARMONIC          float64       
 10  AVG_LENGTH                  float64       
 11  AVG_SPACING                 fl

In [ ]:
filtered_df.to_csv("./Dataset/filtered_data.csv", index=False)
# filtered_df.to_parquet('../Dados/filtered_data.parquet', index=False)

# Transform data for MAB

## Eliminate useless columns

In [43]:
# concatenated AEDL data
df = pd.read_csv("./Dataset/filtered_data.csv", parse_dates=["AGG_PERIOD_START"])
# df = filtered_df.copy()

In [6]:
half_complete_df = df.drop(columns=["AGGREGATE_BY_LANE_BUNDLEID","AGG_ID","AGG_PERIOD_LEN_MINS","NR_LANES","AVG_LENGTH","OCCUPANCY","VOLUME_CLASSE_A","VOLUME_CLASSE_B","VOLUME_CLASSE_C","VOLUME_CLASSE_D","VOLUME_CLASSE_0","AXLE_CLASS_VOLUMES"])
half_complete_df.head(3)
half_complete_df.to_csv("./Dataset/more_complete_data.csv", index=False)
# half_complete_df.to_csv("../Dados/more_complete_data.csv", index=False)
simple_df = half_complete_df.drop(columns=["AVG_SPEED_ARITHMETIC","AVG_SPEED_HARMONIC","AVG_SPACING","LIGHT_VEHICLE_RATE"])
simple_df.head(3)
simple_df.to_csv("./Dataset/simplest_data.csv", index=False)
# simple_df.to_csv("../Dados/simplest_data.csv", index=False)

## Create tags for arms
### (gather all data from the same sensor, the same time interval and the same direction)

In [7]:
# MAP DATA INTO BINS FOR EACH ARM
# Arm = (time_bin, sensor_id, direction)

# Minutes since midnight
simple_df["min_since_midnight"] = (simple_df["AGG_PERIOD_START"].dt.hour*60 + simple_df["AGG_PERIOD_START"].dt.minute)


# 30‑minute bins: [0..47]
simple_df["half_time_bin"] = (simple_df["min_since_midnight"] // 30).astype("int8")

# 5‑minute bins: [0..287]
simple_df["simple_time_bin"] = (simple_df["min_since_midnight"] // 5).astype("int16")

# This maps 00:00–00:04 → bin 0, 00:05–00:09 → bin 1, …, 23:55–23:59 → bin 287, across all days.
# Grouping by such bins is a standard way to compare flows at the same time‑of‑day across dates.

# Define arm identifiers (5-min bin, sensor, direction) as the arm key
simple_df["simple_arm"] = list(
    zip(
        simple_df["simple_time_bin"],
        simple_df["EQUIPMENTID"],
        simple_df["LANE_BUNDLE_DIRECTION"]
    )
)

# Define arm identifiers: (30‑min bin, sensor, direction)
simple_df["half_arm"] = list(
    zip(
        simple_df["half_time_bin"],
        simple_df["EQUIPMENTID"],
        simple_df["LANE_BUNDLE_DIRECTION"]
    )
)

In [8]:
simple_df["half_arm"].nunique()

2496

In [9]:
simple_df["simple_arm"].nunique()

14976

In [10]:
simple_df

,EQUIPMENTID,AGG_PERIOD_START,LANE_BUNDLE_DIRECTION,TOTAL_VOLUME,min_since_midnight,half_time_bin,simple_time_bin,simple_arm,half_arm
0,121749,2015-07-24 08:05:00,D,0,485,16,97,"(97, 121749, D)","(16, 121749, D)"
1,121749,2015-07-24 08:10:00,C,243,490,16,98,"(98, 121749, C)","(16, 121749, C)"
2,121749,2015-07-24 08:10:00,D,0,490,16,98,"(98, 121749, D)","(16, 121749, D)"
3,121749,2015-07-24 08:15:00,C,244,495,16,99,"(99, 121749, C)","(16, 121749, C)"
4,121749,2015-07-24 08:15:00,D,0,495,16,99,"(99, 121749, D)","(16, 121749, D)"
...,...,...,...,...,...,...,...,...,...
3881461,121741,2015-12-31 15:55:00,C,297,955,31,191,"(191, 121741, C)","(31, 121741, C)"
3881462,121741,2015-12-31 15:55:00,D,359,955,31,191,"(191, 121741, D)","(31, 121741, D)"
3881463,121741,2015-12-31 16:00:00,C,266,960,32,192,"(192, 121741, C)","(32, 121741, C)"
3881464,121741,2015-12-31 16:00:00,D,390,960,32,192,"(192, 121741, D)","(32, 121741, D)"


In [11]:
simple_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3881466 entries, 0 to 3881465
Data columns (total 9 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   EQUIPMENTID            int64         
 1   AGG_PERIOD_START       datetime64[ns]
 2   LANE_BUNDLE_DIRECTION  string        
 3   TOTAL_VOLUME           int64         
 4   min_since_midnight     int32         
 5   half_time_bin          int8          
 6   simple_time_bin        int16         
 7   simple_arm             object        
 8   half_arm               object        
dtypes: datetime64[ns](1), int16(1), int32(1), int64(2), int8(1), object(2), string(1)
memory usage: 203.6+ MB


# Train/Test Split

In [ ]:
# Separate the train and test data
cutoff_date = pd.Timestamp("2015-01-01")

# Train data: 2013-2014
simple_train_df = simple_df[simple_df["AGG_PERIOD_START"] < cutoff_date]
# Test data: 2015
simple_test_df  = simple_df[simple_df["AGG_PERIOD_START"] >= cutoff_date]

# Save the data
simple_train_df.to_csv("./Dataset/simple_train.csv", index=False)
simple_test_df.to_csv("./Dataset/simple_test.csv", index=False)
# simple_train_df.to_csv("../Dados/simple_train.csv", index=False)
# simple_test_df.to_csv("../Dados/simple_test.csv", index=False)

In [16]:
simple_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   EQUIPMENTID            0 non-null      int64         
 1   AGG_PERIOD_START       0 non-null      datetime64[ns]
 2   LANE_BUNDLE_DIRECTION  0 non-null      string        
 3   TOTAL_VOLUME           0 non-null      int64         
 4   min_since_midnight     0 non-null      int32         
 5   half_time_bin          0 non-null      int8          
 6   simple_time_bin        0 non-null      int16         
 7   simple_arm             0 non-null      object        
 8   half_arm               0 non-null      object        
dtypes: datetime64[ns](1), int16(1), int32(1), int64(2), int8(1), object(2), string(1)
memory usage: 0.0+ bytes


In [17]:
simple_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3881466 entries, 0 to 3881465
Data columns (total 9 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   EQUIPMENTID            int64         
 1   AGG_PERIOD_START       datetime64[ns]
 2   LANE_BUNDLE_DIRECTION  string        
 3   TOTAL_VOLUME           int64         
 4   min_since_midnight     int32         
 5   half_time_bin          int8          
 6   simple_time_bin        int16         
 7   simple_arm             object        
 8   half_arm               object        
dtypes: datetime64[ns](1), int16(1), int32(1), int64(2), int8(1), object(2), string(1)
memory usage: 203.6+ MB
